In [66]:
# This code read text from numbeo website which collects data from users.

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

city =['Mumbai', 'Delhi', 'Bangalore', 'Hyderabad' , 'Ahmedabad' , 'Chennai', 'Kolkata', 'Surat', 'Pune',
            'Jaipur', 'Kanpur', 'Nagpur', 'Indore', 'Thane', 'Bhopal', 'Patna', "Vadodara"]

goods=['Milk', 'Bread', 'Rice', 'Eggs', 'Cheese', 'Chicken', 'Apples', 'Banana', 'Oranges', 'Tomato','Potato',
       'Onion', 'Lettuce', 'Water (1.5 liter bottle)']

good_and_units = ['Milk (regular) 1 litre' , 'Loaf of fresh white bread (500 gr)' , 'Rice (white), 1 kg', 'Eggs (regular), 12',
             'Local cheese (1 kg)' , 'Chicken Fillets (1 kg)' , 'Apple (1 kg)','Banana (1 kg)',
                 'Orange (1 kg)', 'Tomato (1 kg)', 'Potato (1 kg)', 'Onion (1 kg)', 'Lettuce ( 1 head)','Water bottle (1.5 litre)']

df=pd.DataFrame()
df['items'] = good_and_units

for i in range(len(city)):
    
    #Since some websites do not allow for the crawlers to read their content, we should use proxy.
    proxies = proxies = {
    "http": 'http://200.255.122.170:8080', "https": 'http://83.97.23.90:18080'}
    

    output = ''
    x=requests.get('https://www.numbeo.com/cost-of-living/in/'+ city[i], timeout=10000,proxies=proxies, verify=False) 
    html=x.content
    soup=BeautifulSoup(html,'html.parser') 
    text = soup.find_all(text=True)
    blacklist = ['[document]','noscript','header','html','meta','head','input','script']
    # there may be more elements you don't want, such as "style", etc.

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
            
    output = output.split(sep='\n') # Split the text by its new lines.
    price=[]
    for k in range(len(output)):
        for j in range(len(goods)):
            if goods[j] in output[k]:
                s= re.findall(r"[-+]?\d*\.\d+|\d+", output[k])[-1]
                s=float(s)
                price.append(s)
    price=price[0:14]
    df[city[i]]=price


C:\Users\MICROTECH\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\MICROTECH\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\MICROTECH\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\MICROTECH\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning

In [67]:
#Save the file

writer = pd.ExcelWriter('Price.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()